## Form converter attempt

In [ ]:
const domain = Deno?.args?.[1] || 'localhost';
const CONFIG = `http://${domain.includes('localhost') ? `${domain}:2021` : `config.${domain}`}`;
const GATEWAY = `http://${domain.includes('localhost') ? `${domain}:7070` : `gateway.${domain}`}`;
const COUNTRY_CONFIG = `http://${domain.includes('localhost') ? `${domain}:3040` : `gateway.${domain}`}`;


### Authenticate

In [ ]:
import { authenticate } from './authentication.ts';
const token = await authenticate(GATEWAY, 'k.mweene', 'test');
token

### Fetch v1 forms

In [ ]:
const response = await fetch(`${CONFIG}/forms`, {
  method: 'GET',
  headers: {
    'Content-Type': 'application/json',
    Authorization: `Bearer ${token}`,
  },
})
  if (!response.ok) {
    throw new Error(`Form fetching: ${response.statusText}`);
  }

  const forms  = await response.json()
  forms

### Fetch v2 events

In [ ]:
const response = await fetch(`${COUNTRY_CONFIG}/events`, {
  method: 'GET',
  headers: {
    'Content-Type': 'application/json',
    Authorization: `Bearer ${token}`,
  },
})
if (!response.ok) {
  throw new Error(`Events fetching: ${response.statusText}`)
}

const events = await response.json()
const birthEvent = events.find(e => e.id === 'v2.birth')
birthEvent

### Extract all fields

In [ ]:
function extractFieldType(obj, fieldName) {
  const fields = [];

  function recurse(value) {
    if (Array.isArray(value)) {
      value.forEach(item => recurse(item));
    } else if (value !== null && typeof value === 'object') {
      for (const [key, val] of Object.entries(value)) {
        if (key === fieldName) {
          fields.push(val);
        }
        recurse(val);
      }
    }
  }

  recurse(obj);
  return fields;
}

const labels = extractFieldType(birthEvent, 'label')
const items = extractFieldType(birthEvent, 'items').flatMap(x => x)
const titles = extractFieldType(birthEvent, 'title')
const all = [...labels, ...items, ...titles]

const jsonSet = new Set(all.map(obj => JSON.stringify(obj)));
const matchers = Array.from(jsonSet).map(str => JSON.parse(str));

### Create a rough v1 to v2 form mapping

This pipeline attempts to create a v2 event based on the current v1 form.

While partially successful, it actually had a better use of finding the mappings for things like field labels, options etc.

I ran this for several iterations using the moving the `remapped` id's into `fieldMapper.ts` and then manually diffing the output of `v1-v2.json` with the actual v2 event and adding more mappings.

Any fields not able to be mapped get output at the end.

Some usage ideas:

The mappings in `fieldMapper.ts` can be used to autogenerate translations by inputting the client.csv

The `unmapped` output can be used to see the differences in the two forms and find any missing or ambiguous mappings


In [ ]:
import {
  MAPPING,
  POSTFIX_MAP,
  MAPPING_FOR_CUSTOM_FIELDS,
  LABEL_MAPPING,
} from './fieldMapper.ts?foo=2'
import { template } from './template.ts'

const unmapped = {}
let remapped = {}

const mapId = (field, type) => {
  const filters = [
    (f) => f.id === `v2.${field.id}`,
    (x) =>
      x.description &&
      (x.description === field.description ||
        x.defaultMessage == field.defaultMessage),
    (x) =>
      x.description === field.description &&
      x.defaultMessage == field.defaultMessage,
    (x) =>
      x.description === field.description &&
      x.defaultMessage == field.defaultMessage &&
      x.id.startsWith('v2'),
  ]

  let mapped =
    MAPPING[field.id] ||
    MAPPING_FOR_CUSTOM_FIELDS[field.id] ||
    LABEL_MAPPING[field.id]

  if (mapped) {
    return mapped
  }
  if (!field.id) {
    console.log(field)
  }

  for (const filter of filters) {
    const candidates = matchers.filter(filter)
    if (candidates.length === 1) {
      if (remapped[field.id] && remapped[field.id] !== candidates[0].id) {
        console.log(remapped[field.id])
      }
      remapped[field.id] = candidates[0].id
      return candidates[0].id
    }
  }

  if (!unmapped[type]?.includes(field.id)) {
    ;(unmapped[type] ??= []).push(field.id)
  }
  return field.id
}

const mapLabel = (field) => {
  const { id, ...rest } = field.label
  return { label: { id: mapId(field.label, 'label'), ...rest } }
}

const mapTitle = (title) => {
  if (!title) return {}
  const { id, ...rest } = title
  return { id: mapId(title, 'title'), ...rest }
}

const isCountrySelect = (field) => {
  return field.options?.resource === 'countries'
}

const mapType = (field) => {
  if (isCountrySelect(field)) return 'COUNTRY'
  switch (field.type) {
    case 'SELECT_WITH_OPTIONS':
    case 'SELECT_WITH_DYNAMIC_OPTIONS':
      return 'SELECT'
    case 'DOCUMENT_UPLOADER_WITH_OPTION':
      return 'FILE_WITH_OPTIONS'
    case 'LOCATION_SEARCH_INPUT':
      return 'FACILITY'
    default:
      return field.type
  }
}

const mapOptions = (field) => {
  return (
    field.options &&
    !isCountrySelect(field) && {
      options: field.options.map((o) => ({
        ...o,
        ...mapLabel(o),
      })),
    }
  )
}

const mapItems = (field) => {
  return (
    field.items && {
      items: field.items.map((i) => {
        const { id, ...rest } = i
        return {
          id: mapId(i, 'item'),
          ...rest,
        }
      }),
    }
  )
}

const mapConfiguration = (field) => {
  const configuration = {
    ...(field.maxLength && { maxLength: field.maxLength }),
    ...(field.postfix && { postfix: POSTFIX_MAP[field.postfix] }),
  }

  return Object.keys(configuration).length ? { configuration } : {}
}

const mapConditionals = (field) => {
  const types = {
    hide: 'SHOW',
    disable: 'ENABLE',
    hideInPreview: 'DISPLAY_ON_REVIEW',
  }

  return (
    field.conditionals?.length > 0 && {
      conditionals: field.conditionals.map((c) => ({
        type: types[c.action],
        conditional: ['TODO: conditional'],
      })),
    }
  )
}

const mapValidators = (field) => {
  return (
    field.validator?.length > 0 && {
      validation: field.validator.map(() => 'TODO: validation'),
    }
  )
}

const pages = forms.birth.sections.map((s) => ({
  id: s.id,
  title: mapTitle(s.title),
  type: s.viewType === 'form' ? 'FORM' : 'VALIDATION',
  fields: s.groups.flatMap((g) =>
    g.fields.map((f) => ({
      id: mapId({ id: `birth.${g.id}.${f.name}` }, 'field'), // TODO align with MAPPING
      ...mapLabel(f),
      ...mapConditionals(f),
      type: mapType(f),
      ...(f.required !== undefined && { required: f.required }),
      ...mapOptions(f),
      ...mapItems(f),
      ...mapConfiguration(f),
      ...mapValidators(f),
    }))
  ),
}))

const toV2 = template
toV2[0].declaration.pages = pages

await Deno.writeTextFile('./v1-v2.json', JSON.stringify(toV2, null, 2))
//await Deno.writeTextFile('./remapped.json', JSON.stringify(remapped, null, 2))

unmapped
